In [1]:
!pip install pandas sentence-transformers faiss-cpu
!pip install tensorflow==2.15.0 keras==2.15.0
! pip install pandas sentence-transformers faiss-cpu scikit-learn matplotlib tqdm


In [55]:
#conda create -n my_env python=3.10
#conda activate my_env
#pip install tensorflow==2.15.0 keras==2.15.0


In [1]:
import pandas as pd

In [33]:
df = pd.read_excel('Data.xlsx')
df.head(5)

,Unnamed: 0,incident_number,application,error_code,microservice,description,detailed_description,resolution_comments
0,0,INC000005184952,NaN,NaN,NaN,NaN,AP:PRD:UR: Authorization has failed\n\nAlert T...,Changed Group assignment to Universal Reposito...
1,1,INC000005183351,DIGITAL_PLATFORM,ServiceException,ur-api-gateway,Cassandra timeout during read query at consist...,Stage:prod\nMicroservice:\tur-api-gateway\nTra...,Changed Group assignment to Universal Reposito...
2,2,INC000005183248,DIGITAL_PLATFORM,ServiceException,ur-api-gateway,Cassandra timeout during read query at consist...,Stage:prod\nMicroservice:\tur-api-gateway\nTra...,Changed Group assignment to Universal Reposito...
3,3,INC000005183246,DIGITAL_PLATFORM,ServiceException,ur-api-gateway,Cassandra timeout during read query at consist...,Stage:prod\nMicroservice:\tur-api-gateway\nTra...,Changed Group assignment to Universal Reposito...
4,4,INC000005180860,Customer Communications->Digital->Presentment ...,NaN,NaN,NaN,Summary of the issue: ICMA - removal of digita...,Changed Group assignment to EDH CSM9 Assigned ...


In [40]:
incident_dict = df.set_index('incident_number')['resolution_comments'].to_dict()


In [41]:
df.set_index('incident_number', inplace=True)

In [42]:
# Concatenate relevant columns into a single string for embedding
df['text'] = df[['application', 'error_code', 'description', 'detailed_description']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)


In [43]:
data = df.copy()

In [44]:
data = data[['text']][:10]

In [45]:
data.shape

(10, 1)

In [46]:
data

,text
incident_number,
INC000005184952,AP:PRD:UR: Authorization has failed\n\nAlert T...
INC000005183351,DIGITAL_PLATFORM ServiceException Cassandra ti...
INC000005183248,DIGITAL_PLATFORM ServiceException Cassandra ti...
INC000005183246,DIGITAL_PLATFORM ServiceException Cassandra ti...
INC000005180860,Customer Communications->Digital->Presentment ...
INC000005180399,DIGITAL_PLATFORM BENGAL_TIGER (TraceId=65b7ced...
INC000005180243,DIGITAL_PLATFORM BENGAL_TIGER (TraceId=65b7cec...
INC000005178679,DIGITAL_PLATFORM ReplyException Timed out afte...
INC000005178635,DIGITAL_PLATFORM ServiceException operation ti...


In [47]:
data.text[4]

C:\Users\rc11692\AppData\Local\Temp\ipykernel_31620\2578418679.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data.text[4]


'Customer Communications->Digital->Presentment (Including Legacy PostEdge) Summary of the issue: ICMA - removal of digital onserts for 1099 tax forms\nDescription of the issue: ICMA is requesting we remove the digital onsert from the forms as it is pulling in the previous year’s form. (Due to time constraints, this year’s form was not included but the previous year’s is still being pulled in)\nSeverity: Sev3\nClients Affected: Single\nClients Affected (Single): ICMA\nClient number (Single): I00022\nActual Incident Start Time: 1/25/2024 9:00 AM\nBusiness Impact?: Yes\nBusiness Impact: Yes \nClient Impact: Yes \nBusiness Unit: BRCC\nAffected Application: Customer Communications->Digital->Presentment (Including Legacy PostEdge)\nIdentified Source: Client Detected\nEnvironment: Production\n'

In [48]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(data['text'].tolist(), show_progress_bar=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [49]:
embeddings

array([[-0.07375074,  0.06568636, -0.04863684, ..., -0.04706039,
        -0.08331089,  0.066782  ],
       [-0.02130117, -0.03041642, -0.0099752 , ..., -0.04174381,
        -0.0460832 , -0.00206137],
       [-0.00408928, -0.03323755, -0.00500118, ..., -0.03572897,
        -0.06078001, -0.00911633],
       ...,
       [-0.04008536, -0.01256368, -0.01236878, ...,  0.00593633,
        -0.07521831, -0.02346144],
       [-0.01994997, -0.00201161, -0.03769414, ...,  0.02792372,
        -0.03477802, -0.024237  ],
       [-0.01687484, -0.00313686, -0.03950549, ...,  0.02572631,
        -0.03553012, -0.0249915 ]], dtype=float32)

In [50]:
import faiss
import numpy as np

# Convert embeddings to a numpy array
embeddings = np.array(embeddings)

# Initialize Faiss index
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)

# Add embeddings to the index
index.add(embeddings)

# Save the index to disk
faiss.write_index(index, 'incident_index.faiss')


In [54]:
def search_similar_incidents(query, top_n=1):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), top_n)
    
    results = []
    for idx in I[0]:
        incident_id = df.index[idx]
        results.append((incident_id, df.loc[incident_id]['text']))
    
    return results

# Example usage:
query = "Summary of the issue: ICMA - removal of digital onserts for 1099 tax forms\nDescription of the issue: ICMA is requesting we remove the digital onsert f"
similar_incidents = search_similar_incidents(query)
for incident_id, text in similar_incidents:
    print(f"resolution of above one is {incident_dict[incident_id]}")
    #print(f"Incident ID: {incident_id}\nText: {text}\n")


resolution of above one is Changed Group assignment to EDH CSM9 Assigned To: givensr by givensr Changed Group assignment to EDH INCIDENT MGMT Per Casie Thompson, this is a duplicate - the issue has already been taken care of. This ticket can be closed. 
***MIR3 PAGE RESPONSE***
From: Gautam Ramalingam 
Selected Response: Accept 
At: 1/29/2024 5:41:44 PM EST

https://remtieritsm.broadridge.net/arsys/forms/remproditsm/BR%3AMIR3%3ASearchUI?F536870975=1&F536870976=MIR300001090759

https://webnotify.mir3.com/prepareEventHist.do?eventHistId=3e76c3f4-0003-3000-80c0-fceb55463ffe ***MIR3 PAGE TRIGGERED*** 
Sent to: BRCC UR Core team 
At 1/29/2024 5:36:12 PM EST
Message: Please review and address incident ICMA - removal of digital onserts for 1099 tax forms. Please reference incident number 5180860

 Incident Communication Channel: 

Page sent by: Richard J Givens
https://smartit.broadridge.net/smartit/app/#/incident/IDGAA5V0H1S9XASIBMBHSHBC0GYUYY
Note: Incident link works only on BR network
htt

In [ ]:
# Save local and use 

In [56]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load your CSV data
#df = pd.read_csv('your_file.csv')
df = pd.read_excel('Data.xlsx')
df['text'] = df[['application', 'error_code', 'description', 'detailed_description']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df.set_index('incident_number', inplace=True)

df = df[:5]

# Generate embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)
embeddings = np.array(embeddings)

# Initialize Faiss index
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)

# Add embeddings to the index
index.add(embeddings)

# Save the index to disk
faiss.write_index(index, 'incident_index.faiss')

# Save the DataFrame to disk
df.to_csv('incident_data.csv')


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the Faiss index from disk
index = faiss.read_index('incident_index.faiss')

# Load the DataFrame from disk
df = pd.read_csv('incident_data.csv', index_col='incident_number')

# Initialize the model for embedding queries
model = SentenceTransformer('all-MiniLM-L6-v2')

def search_similar_incidents(query, top_n=3):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), top_n)
    
    results = []
    for idx in I[0]:
        incident_id = df.index[idx]
        results.append((incident_id, df.loc[incident_id]['text']))
    
    return results

# Example usage:
query = "Cassandra timeout during read query"
similar_incidents = search_similar_incidents(query)
for incident_id, text in similar_incidents:
    print(f"Incident ID: {incident_id}\nText: {text}\n")


Incident ID: INC000005183248
Text: DIGITAL_PLATFORM ServiceException Cassandra timeout during read query at consistency LOCAL_QUORUM (2 responses were required but only 0 replica responded). In case this was generated during read repair, the consistency level is not representative of the actual consistency. Stage:prod
Microservice:	ur-api-gateway
Trace Data:				(TraceId=65b94c59cb6c3e2b1726523df5e2289c, ParentSpanId=d1d9e5993a880da5, SpanId=b25055601044fa67)
Application:		DIGITAL_PLATFORM

User Name:		dp_prod

Error Code:		ServiceException

Description:		Cassandra timeout during read query at consistency LOCAL_QUORUM (2 responses were required but only 0 replica responded). In case this was generated during read repair, the consistency level is not representative of the actual consistency.

Incident ID: INC000005183246
Text: DIGITAL_PLATFORM ServiceException Cassandra timeout during read query at consistency LOCAL_QUORUM (2 responses were required but only 0 replica responded). In case

In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV data
df = pd.read_excel('Data.xlsx')

df = df

# Concatenate relevant columns into a single string for embedding
df['text'] = df[['application', 'error_code', 'description', 'detailed_description']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Set the incident_number as the index for both
train_df.set_index('incident_number', inplace=True)
val_df.set_index('incident_number', inplace=True)


In [66]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# Initialize the model for embedding queries
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the training set
train_embeddings = model.encode(train_df['text'].tolist(), show_progress_bar=True)
train_embeddings = np.array(train_embeddings)

# Normalize the embeddings for cosine similarity
faiss.normalize_L2(train_embeddings)

# Initialize Faiss index
d = train_embeddings.shape[1]
index = faiss.IndexFlatIP(d)  # IP stands for Inner Product which is equivalent to cosine similarity on normalized vectors

# Add embeddings to the index
index.add(train_embeddings)

# Create incident dictionary
incident_dict = train_df['resolution_comments'].to_dict()

# Save the index and incident dictionary
faiss.write_index(index, 'incident_index_cosine.faiss')
import pickle
with open('incident_dict.pkl', 'wb') as f:
    pickle.dump(incident_dict, f)


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

In [67]:
import pickle
from sklearn.metrics import accuracy_score

# Load the Faiss index and incident dictionary
index = faiss.read_index('incident_index_cosine.faiss')
with open('incident_dict.pkl', 'rb') as f:
    incident_dict = pickle.load(f)

# Generate embeddings for the validation set
val_embeddings = model.encode(val_df['text'].tolist(), show_progress_bar=True)
faiss.normalize_L2(val_embeddings)

# Perform similarity search and evaluate
def search_similar_incidents(query_embedding, top_n=1):
    D, I = index.search(np.array([query_embedding]), top_n)
    results = []
    for idx in I[0]:
        incident_id = train_df.index[idx]
        results.append((incident_id, train_df.loc[incident_id]['text']))
    return results

# Calculate accuracy
correct = 0
for i, row in val_df.iterrows():
    query_embedding = model.encode([row['text']])[0]
    result = search_similar_incidents(query_embedding, top_n=1)
    if result and incident_dict[result[0][0]] == row['resolution_comments']:
        correct += 1



Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [68]:
accuracy = correct / len(val_df)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5


In [ ]:
#last exp

In [69]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

# Load the CSV data
df = pd.read_excel('Data.xlsx')

# Concatenate relevant columns into a single string for embedding
df['text'] = df[['application', 'error_code', 'description', 'detailed_description']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df.set_index('incident_number', inplace=True)

# Generate embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)
embeddings = np.array(embeddings)

# Normalize the embeddings for cosine similarity
faiss.normalize_L2(embeddings)

# Initialize Faiss index
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)  # IP stands for Inner Product which is equivalent to cosine similarity on normalized vectors

# Add embeddings to the index
index.add(embeddings)

# Save the index and DataFrame
faiss.write_index(index, 'incident_index_cosine.faiss')
df.to_csv('incident_data.csv')
incident_dict = df['resolution_comments'].to_dict()
with open('incident_dict.pkl', 'wb') as f:
    pickle.dump(incident_dict, f)


Batches:   0%|          | 0/99 [00:00<?, ?it/s]

In [70]:
# Select a sample for validation
val_df = df.sample(frac=0.2, random_state=42)  # Using 20% of the data for validation

# Drop the selected validation samples from the original DataFrame to simulate a training dataset
train_df = df.drop(val_df.index)

# Reduce the text data length for validation
val_df['text'] = val_df['text'].apply(lambda x: ' '.join(x.split()[:50]))  # Using first 50 words as an example


In [71]:
# Load the Faiss index and incident dictionary
index = faiss.read_index('incident_index_cosine.faiss')
df = pd.read_csv('incident_data.csv', index_col='incident_number')
with open('incident_dict.pkl', 'rb') as f:
    incident_dict = pickle.load(f)

def search_similar_incidents(query_embedding, top_n=1):
    D, I = index.search(np.array([query_embedding]), top_n)
    results = []
    for idx in I[0]:
        incident_id = df.index[idx]
        results.append((incident_id, df.loc[incident_id]['text']))
    return results

# Calculate accuracy on the validation set
correct = 0
for i, row in val_df.iterrows():
    query_embedding = model.encode([row['text']])[0]
    result = search_similar_incidents(query_embedding, top_n=1)
    if result and incident_dict[result[0][0]] == row['resolution_comments']:
        correct += 1

accuracy = correct / len(val_df)
print(f"Accuracy: {accuracy}")


Accuracy: 0.8821656050955414


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, classification_report, precision_recall_curve, accuracy_score

from tqdm import tqdm

# Load the Faiss index and incident dictionary
index = faiss.read_index('incident_index_cosine.faiss')
df = pd.read_csv('incident_data.csv', index_col='incident_number')
with open('incident_dict.pkl', 'rb') as f:
    incident_dict = pickle.load(f)

def search_similar_incidents(query_embedding, top_n=1):
    D, I = index.search(np.array([query_embedding]), top_n)
    results = []
    for idx in I[0]:
        incident_id = df.index[idx]
        results.append((incident_id, df.loc[incident_id]['text']))
    return results

# Calculate precision, recall, and accuracy on the validation set
y_true = []
y_pred = []

correct_predictions = []
wrong_predictions = []

for i, row in tqdm(val_df.iterrows(), total=val_df.shape[0], desc="Processing validation set"):
    query_embedding = model.encode([row['text']])[0]
    result = search_similar_incidents(query_embedding, top_n=1)
    true_label = row['resolution_comments']
    y_true.append(true_label)
    if result:
        predicted_label = incident_dict[result[0][0]]
        y_pred.append(predicted_label)
        if true_label == predicted_label:
            correct_predictions.append((i, row['text'], true_label))
        else:
            wrong_predictions.append((i, row['text'], true_label, predicted_label))
    else:
        y_pred.append("")
        wrong_predictions.append((i, row['text'], true_label, ""))

# Convert lists to numpy arrays for evaluation
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate precision, recall, and f1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='micro')
accuracy = accuracy_score(y_true, y_pred)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_true, y_pred))

# Create DataFrames for correct and wrong predictions
correct_df = pd.DataFrame(correct_predictions, columns=['incident_id', 'text', 'true_label'])
wrong_df = pd.DataFrame(wrong_predictions, columns=['incident_id', 'text', 'true_label', 'predicted_label'])

# Display DataFrames
correct_df.to_csv('correct_predictions.csv', index=False)
wrong_df.to_csv('wrong_predictions.csv', index=False)

print("Correct Predictions DataFrame:")
print(correct_df.head())

print("Wrong Predictions DataFrame:")
print(wrong_df.head())

# Plot Precision-Recall curve
precision_vals, recall_vals, _ = precision_recall_curve(y_true, y_pred, pos_label=1)

plt.figure()
plt.plot(recall_vals, precision_vals, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()



Processing validation set: 100%|██████████████████████████████████████████████████████████████████| 628/628 [04:32<00:00,  2.31it/s]


Precision: 0.8821656050955414
Recall: 0.8821656050955414
F1-score: 0.8821656050955414
Accuracy: 0.8821656050955414
